<a href="https://colab.research.google.com/github/MinuMaryS/Coursera_Capstone/blob/main/CourseraCapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np


<h3>This is my final capstone project document. 

The aim of the project is to analyze neighbourhoods in Toronto.


In [23]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [24]:
from bs4 import BeautifulSoup
from urllib.request import urlopen


In [25]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df=pd.read_html(url,header=0)

Since there are multiple tables in the dataframe, we select onlythe first table containing postal codes for our project.

In [26]:
postalcodes=df[0]

In [27]:
postalcodes

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [28]:
postalcodes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


Since 'Not Assigned' is not considered as null value by default, we assign it null value and then drop those rows.


In [29]:
postalcodes['Borough']=np.where(postalcodes['Borough']=='Not assigned',np.nan,postalcodes['Borough'])

In [30]:
postalcodes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [31]:
postalcodes.dropna(inplace=True)

In [32]:
postalcodes.head(20)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [33]:
postalcodes['Neighbourhood']=np.where(postalcodes['Neighbourhood']=='Not assigned',postalcodes['Borough'],postalcodes['Neighbourhood'])